In [ ]:
PROJECT_ID = "PROJECT_ID"
DATASET_ID = "hackathon"

SERVICE_ACCOUNT = "SERVICE_ACCOUNT"

import os

GOOGLE_APPLICATION_CREDENTIALS = "../../credentials.json"  # @param {type:"string"}
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = GOOGLE_APPLICATION_CREDENTIALS

!gcloud config set account {SERVICE_ACCOUNT}
!gcloud auth activate-service-account --key-file={GOOGLE_APPLICATION_CREDENTIALS}
!gcloud config set project {PROJECT_ID} 

In [ ]:
from google.cloud import bigquery
import os
import glob

In [ ]:
client = bigquery.Client()

In [ ]:
dataset = bigquery.Dataset(f"{PROJECT_ID}.{DATASET_ID}")
dataset.location = "EU"
dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

In [ ]:
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV, skip_leading_rows=1, autodetect=True,
)

In [ ]:
for filename in glob.glob("*.csv"):
    
    table_name = filename.split(".")[0]
    table_id = f"{PROJECT_ID}.{DATASET_ID}.{table_name}"
    
    with open(filename, "rb") as source_file:
        job = client.load_table_from_file(source_file, table_id, job_config=job_config)
    
    job.result()
    
    table = client.get_table(table_id)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )